## **Mounting the Drive**

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install transformers
!pip3 install unidecode
!pip install pandas
!pip install keras 
!pip install tensorflow
!pip install torch
!pip install git+https://github.com/AndriyMulyar/semantic-text-similarity
!pip install urllib3==1.25.10
!pip install awscli awsebcli botocore==1.18.18 --upgrade
!pip install tqdm

In [ ]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import join
import unidecode
import re
import logging
from tqdm.notebook import tnrange
import glob
import json
import os

# Creating Corpus for Pyserini

In [ ]:
def generate_corpus(df,  start , end):
    new_df = pd.DataFrame(columns=['question','sentence'])

    pd.set_option('display.max_seq_items', None)
    num_psges =  0

    for i in tnrange(start,end):
        cntxt_len = len(df['context'][i])
        answer =  df['answer'][i]
        # question = str(df['question'][i] +" "+answer) 
        question = df['question'][i]
        num_psges  = num_psges + cntxt_len
        if len(question) < 2:
            continue

        for j  in range(0,cntxt_len):
            passage  =  "".join(df['context'][i][j][1])

            senList  = df['context'][i][j][1]
            # passage_title =  df['context'][i][j][0]
            for each_sen in senList:
                # filtering empty sentences
                if len(each_sen) < 2 :
                    continue
            new_row = {'question' : question, 'sentence': passage}
            new_df =  new_df.append(new_row,ignore_index= True)
    return  new_df
   

In [ ]:
start = 0
end = 7405
FOLDER = '/content/drive/My Drive/Courses/NLP/Project/data'
JSON_FILE =  FOLDER +"hotpot_dev_distractor_v1.json"
df = pd.read_json('/content/drive/MyDrive/Courses/NLP/Project/data/hotpot_dev_distractor_v1.json')
dev_df  =  generate_corpus(df,start,end)
dev_df.to_csv('/content/drive/MyDrive/Courses/NLP/Project/data/dev_0_7405.csv', header=True, index=False)

In [ ]:
load_data = '/content/drive/My Drive/Courses/NLP/Project/data'
train_df = pd.read_csv(join(load_data,'result_exact_ans_0_5000.csv'))

#Train Passages
train_passages = pd.DataFrame(columns=['sentence'])
train_passages['sentence'] = train_df['sentence']

#Dev Passages
dev_passages = pd.DataFrame(columns=['sentence'])
dev_passages['sentence'] = dev_df['sentence']

#total  corpus
corpus_df = dev_passages.append(train_passages, ignore_index=True)
corpus_df.to_csv('/content/drive/MyDrive/Courses/NLP/Project/data/total_corpus.tsv', header=False, index=True, sep='\t')



In [ ]:
print(len(corpus_df))

123219


In [ ]:
train_passages.head()

In [ ]:
!pip install pyserini==0.9.4.0

     |████████████████████████████████| 60.4MB 60kB/s 
     |████████████████████████████████| 1.1MB 40.7MB/s 


In [ ]:
%cd /content/drive/MyDrive/Courses/NLP/Project/data/HOTPOTQA_Corpus/anserini

/content/drive/MyDrive/Courses/NLP/Project/data/HOTPOTQA_Corpus/anserini


In [ ]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"     #set environment variable
  !java -version     #check java version
install_java()

openjdk version "11.0.9.1" 2020-11-04
OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)


# **Clone the Pyserini Repository**

In [ ]:
!git clone https://github.com/castorini/pyserini.git

Cloning into 'pyserini'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 1367 (delta 22), reused 24 (delta 12), pack-reused 1318
Receiving objects: 100% (1367/1367), 430.04 KiB | 1.78 MiB/s, done.
Resolving deltas: 100% (866/866), done.


In [ ]:
%cd pyserini
!git clone https://github.com/castorini/anserini-tools.git
%cd /content/drive/MyDrive/Courses/NLP/Project/data/HOTPOTQA_Corpus/anserini

/content/drive/MyDrive/Courses/NLP/Project/data/HOTPOTQA_Corpus/anserini/pyserini
fatal: destination path 'anserini-tools' already exists and is not an empty directory.
/content/drive/MyDrive/Courses/NLP/Project/data/HOTPOTQA_Corpus/anserini


In [ ]:
sentences= []
index=[]
i=1
corpus_sentences=open('/content/drive/My Drive/QASC-DATASET/data/QASC_Corpus/QASC_Corpus.txt','r')
for line in corpus_sentences.readlines():
    lin = line.replace('\n', ' ')
    index.append(i)
    sentences.append(lin)
    i=i+1
  

In [ ]:
print(index[100000])


100001


In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(index, sentences)), columns=['index','paragraph'])
df.to_csv('/content/drive/My Drive/QASC-DATASET/data/QASC_Corpus/anserini/pyserini/collections/document_passages.tsv', header=False, index=False, sep='\t')

**Loading input Corpus dataset in TSV format and converting it into JSONL with each file containing max 1000000(1M) Sentences**

In [ ]:
df_data= pd.read_csv('/content/drive/My Drive/QASC-DATASET/data/QASC_Corpus/anserini/pyserini/collections/document_passages.tsv', sep='\t')

In [ ]:
!python pyserini/anserini-tools/scripts/msmarco/convert_collection_to_jsonl.py \
 --collection-path pyserini/collections/total_corpus.tsv \
 --output-folder pyserini/collections/collection_jsonl

**Creating JSON collections for the corpus**

In [ ]:
!python -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator \
 -threads 9 -input pyserini/collections/collection_jsonl \
 -index pyserini/indexes/lucene-index-hotpotqa -storePositions -storeDocvectors -storeRaw

2020-11-28 07:09:11,024 INFO  [main] index.IndexCollection (IndexCollection.java:636) - Setting log level to INFO
2020-11-28 07:09:11,026 INFO  [main] index.IndexCollection (IndexCollection.java:639) - Starting indexer...
2020-11-28 07:09:11,026 INFO  [main] index.IndexCollection (IndexCollection.java:640) - ============ Loading Parameters ============
2020-11-28 07:09:11,027 INFO  [main] index.IndexCollection (IndexCollection.java:641) - DocumentCollection path: pyserini/collections/collection_jsonl
2020-11-28 07:09:11,027 INFO  [main] index.IndexCollection (IndexCollection.java:642) - CollectionClass: JsonCollection
2020-11-28 07:09:11,028 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Generator: DefaultLuceneDocumentGenerator
2020-11-28 07:09:11,028 INFO  [main] index.IndexCollection (IndexCollection.java:644) - Threads: 9
2020-11-28 07:09:11,029 INFO  [main] index.IndexCollection (IndexCollection.java:645) - Stemmer: porter
2020-11-28 07:09:11,029 INFO  [main] inde

**Sample Testing of the indexing done by the Pyserini**

In [ ]:
from pyserini.search import SimpleSearcher
import pandas as pd
from os.path import join
import numpy as np

searcher = SimpleSearcher('pyserini/indexes/lucene-index-hotpotqa')

searcher.set_bm25(1.2, 0.75)

hits = searcher.search("American biographical period",  k = 50)

# Print the first 10 hits:
for hit in hits:
    print(f'{hit.docid:15} {hit.score:.5f}')

0               6.02080
45008           6.02080
26855           5.37480
86752           5.19280
57381           4.88170
33114           4.79810
21775           4.71940
42463           4.62650
110503          4.60190
82200           4.50390
81556           4.34680
82849           4.33540
106707          4.31080
118489          4.30930
93524           4.30930
22977           4.28860
33887           4.27230
51377           4.27230
57021           4.27230
56825           4.07860
57378           4.06660
57383           4.06340
42379           4.03760
123161          3.98210
9684            3.98210
102078          3.96640
33831           3.96640
65120           3.96640
3227            3.93510
30325           3.92780
28650           3.90430
40922           3.90430
45690           3.90430
13943           3.88800
68522           3.88800
42794           3.88190
30180           3.87400
18462           3.84310
89574           3.82530
104763          3.81470
12224           3.81470
28649           

In [ ]:
d=searcher.doc(str(hits[0].docid))
print(d.raw)

<bound method Document.raw of <pyserini.search._base.Document object at 0x7f3e4616f6d8>>


In [ ]:
print(hits)

[<io.anserini.search.SimpleSearcher$Result at 0x7f3e4c82f830 jclass=io/anserini/search/SimpleSearcher$Result jself=<LocalRef obj=0xf355518 at 0x7f3e3f2879d0>>, <io.anserini.search.SimpleSearcher$Result at 0x7f3e4c82f990 jclass=io/anserini/search/SimpleSearcher$Result jself=<LocalRef obj=0xf355588 at 0x7f3e51903d70>>, <io.anserini.search.SimpleSearcher$Result at 0x7f3e4c82f7d8 jclass=io/anserini/search/SimpleSearcher$Result jself=<LocalRef obj=0xf3555b0 at 0x7f3e44758830>>, <io.anserini.search.SimpleSearcher$Result at 0x7f3e4c82f1a8 jclass=io/anserini/search/SimpleSearcher$Result jself=<LocalRef obj=0xf3555d0 at 0x7f3e44758310>>, <io.anserini.search.SimpleSearcher$Result at 0x7f3e4c82f9e8 jclass=io/anserini/search/SimpleSearcher$Result jself=<LocalRef obj=0xf3555e8 at 0x7f3e44758890>>, <io.anserini.search.SimpleSearcher$Result at 0x7f3e4b1fb048 jclass=io/anserini/search/SimpleSearcher$Result jself=<LocalRef obj=0xf3555f0 at 0x7f3e51903490>>, <io.anserini.search.SimpleSearcher$Result at 

In [ ]:
R=[]
doc = searcher.doc(str(hits[0].docid))
R.append(doc.raw())